# This file lists all the functions required for running the script

In [ ]:
def collect_results(result):
    """Uses apply_async's callback to setup up a separate Queue for each process"""
    results.extend(result)

In [ ]:
def extract_pixel_counts(image_file, intersection_file, map_file, map_coverage, chunk):

    intersection_file = intersection_file[chunk:chunk + 1000]
    
    df = []
    
    with rasterio.open(image_file) as src:

        for i in range(len(intersection_file)):
            
            guf_part, guf_part_t = rasterio.mask.mask(src, intersection_file[intersection_file.intersection_id == intersection_file.iloc[i].intersection_id].geometry,crop=True)
            guf_adm, guf_adm_t = rasterio.mask.mask(src, map_file[map_file.MAP_ID == intersection_file.iloc[i].MAP_ID].geometry,crop=True)
            guf_ant, guf_ant_t = rasterio.mask.mask(src, map_coverage[map_coverage.voronoi_id == intersection_file.iloc[i].voronoi_id].geometry,crop=True)
                
            if np.count_nonzero(guf_adm == 255) == 0:
                w_guf_adm = 0
            else:
                w_guf_adm = np.count_nonzero(guf_part == 255)/np.count_nonzero(guf_adm == 255)

            if np.count_nonzero(guf_ant == 255) == 0:
                w_guf_ant = 0
            else:
                w_guf_ant = np.count_nonzero(guf_part == 255)/np.count_nonzero(guf_ant == 255)
                
            df.append([intersection_file.intersection_id.iloc[i], w_guf_adm, w_guf_ant])

    return df

In [ ]:
def extract_grid_counts(image_file, intersection_file, map_file, map_coverage, chunk):

    intersection_file = intersection_file[chunk:chunk + 1000]
    
    df = []
    
    with rasterio.open(image_file) as src:

        for i in range(len(intersection_file)):
            
            wpg_part, wpg_part_t = rasterio.mask.mask(src, intersection_file[intersection_file.intersection_id == intersection_file.iloc[i].intersection_id].geometry,crop=True)
            wpg_adm, wpg_adm_t = rasterio.mask.mask(src, map_file[map_file.MAP_ID == intersection_file.iloc[i].MAP_ID].geometry,crop=True)
            wpg_ant, wpg_ant_t = rasterio.mask.mask(src, map_coverage[map_coverage.voronoi_id == intersection_file.iloc[i].voronoi_id].geometry,crop=True)
                
            if np.sum(wpg_adm) == 0:
                w_wpg_adm = 0
            else:
                w_wpg_adm = np.sum(wpg_part[wpg_part >= 0])/np.sum(wpg_adm[wpg_adm >= 0])

            if np.sum(wpg_ant) == 0:
                w_wpg_ant = 0
            else:
                w_wpg_ant = np.sum(wpg_part[wpg_part >= 0])/np.sum(wpg_ant[wpg_ant >= 0])
                
            df.append([intersection_file.intersection_id.iloc[i], w_wpg_adm, w_wpg_ant])
            

    return df

In [ ]:
def coordinates_from_rows_and_cols(r,c,gt):
    
    x = gt[1] * c + gt[2] * r + gt[1] * 0.5 + gt[2] * 0.5 + gt[0]
    y = gt[4] * c + gt[5] * r + gt[4] * 0.5 + gt[5] * 0.5 + gt[3]
    
    return(x,y)

In [ ]:
def extract_pixel_information(file, raster_x_size, raster_y_size, step_size):
    
    result = pd.DataFrame(columns = ['geometry', 'value'])
    
    i = raster_x_size

    for j in tqdm(range(0, raster_y_size, step_size)):
        
        # Check if in the range of the raster is any black pixel. If there is none, jump to the next window.
        with rasterio.open(file) as src:
            
            w = src.read(1, window=Window(col_off = i, row_off = j, width = step_size, height = step_size))
                
            if np.count_nonzero(w == 255) == 0:
                
                continue
                
            else:
            
                # Extract coordinates from column and rows
                df = gpd.GeoDataFrame.from_records(itertools.product(range(i,i + step_size),range(j, j + step_size)),columns=['Row','Column'])
                df['X'], df['Y'] = zip(*df.apply(lambda x: coordinates_from_rows_and_cols(r = x['Column'],c = x['Row'],gt = gt),axis=1))
                df['geometry'] = [Point(xy) for xy in zip(df.X, df.Y)]
                df.crs = map_commune.crs

                # Extract pixel values for points
                df['value'] = point_query(df['geometry'], file_guf)

                # Reduce dataset to black pixels only
                df = df[df.value == 255].drop(columns = ['X', 'Y', 'Row', 'Column'])

        result = result.append(df)

    return(result.values.tolist())

In [ ]:
def extract_grid_information(file, raster_x_size, raster_y_size, step_size):
    
    result = pd.DataFrame(columns = ['geometry', 'pop_per_grid'])
    
    i = raster_x_size

    for j in tqdm(range(0, raster_y_size, step_size)):
        
        # Check if in the range of the raster is any black pixel. If there is none, jump to the next window.
        with rasterio.open(file) as src:
            
            w = src.read(1, window=Window(col_off = i, row_off = j, width = step_size, height = step_size))
                
            # Extract coordinates from column and rows
            df = gpd.GeoDataFrame.from_records(itertools.product(range(i,i + step_size),range(j, j + step_size)),columns=['Row','Column'])
            df['X'], df['Y'] = zip(*df.apply(lambda x: coordinates_from_rows_and_cols(r = x['Column'],c = x['Row'],gt = gt),axis=1))
            df['geometry'] = [Point(xy) for xy in zip(df.X, df.Y)]
            df.crs = map_commune.crs

            # Extract pixel values for points
            df['pop_per_grid'] = point_query(df['geometry'], file)

            # Reduce dataset to black pixels only
            df = df[df.pop_per_grid >= 0].drop(columns = ['X', 'Y', 'Row', 'Column'])

        result = result.append(df)

    return(result.values.tolist())

In [ ]:
def ckdnearest(start_points, neighbor_points, k_nearest):
    nA = np.array(list(zip(start_points.geometry.x, start_points.geometry.y)) )
    nB = np.array(list(zip(neighbor_points.geometry.x, neighbor_points.geometry.y)) )
    btree = spatial.cKDTree(nB)
    dist, idx = btree.query(nA, k = [k_nearest])
    gdf = pd.concat(
        [start_points, neighbor_points.loc[idx.flatten(), neighbor_points.columns != 'geometry'].reset_index(),
         pd.Series(dist.flatten(), name = 'distance')], axis=1)
    return gdf

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r